# Parker Solar Probe data

The parameters are:

- 'psp_fld_l2_mag_RTN_4_Sa_per_Cyc' (B in RTN, 0.25s resolution)
- 'psp_swp_spi_sf00_L3_DENS' (Ion density, regular resolution)
- 'psp_swp_spi_af00_L3_DENS' (Ion density, burst resolution)
- 'psp_swp_spi_sf00_L3_VEL_RTN_SUN' (velocity in RTN in Sun's frame, regular resolution)
- 'psp_swp_spi_af00_L3_VEL_RTN_SUN' (velocity in RTN in Sun's frame, burst resolution)
- 'psp_swp_spi_sf00_L3_VEL_RTN' (velocity in RTN in spacecraft frame, regular resolution, you need this to calculate current sheet thickness)
- 'psp_swp_spi_af00_L3_VEL_RTN' (velocity in RTN in spacecraft frame, burst resolution, you need this to calculate current sheet thickness)
- 'Tp_spani_b' (Ion parallel and perpendicular ion temperature, regular resolution. Blue (first element in array): parallel, Red: perp)
- 'Tp_spanib_b' (Ion parallel and perpendicular ion temperature, burst resolution. Blue (first element in array): parallel, Red: perp)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from psp.io import get_data_lf
from psp.io.psp import load_psp_data
from psp.io.psp import PSP_MAG_TNAME, PSP_DEN_TNAME, PSP_VEL_TNAME, PSP_TEMP_TNAME

In [3]:
from discontinuitypy.datasets import IDsDataset
from space_analysis.meta import PlasmaDataset
import numpy as np
from utils.temp import calc_pressure_anisotropy

In [4]:
enc = 7
tau = 16
name = "psp"

In [5]:
load_psp_data(enc=enc)

0 : psp_fld_l2_mag_RTN_4_Sa_per_Cyc
1 : psp_swp_spi_sf00_L3_DENS
2 : psp_swp_spi_sf00_L3_VEL_RTN_SUN
3 : psp_swp_spi_sf00_L3_VEL_RTN
4 : psp_swp_spi_af00_L3_DENS
5 : psp_swp_spi_af00_L3_VEL_RTN_SUN
6 : psp_swp_spi_af00_L3_VEL_RTN
7 : Tp_spani_b
8 : Tp_spanib_b


['psp_fld_l2_mag_RTN_4_Sa_per_Cyc',
 'psp_swp_spi_sf00_L3_DENS',
 'psp_swp_spi_sf00_L3_VEL_RTN_SUN',
 'psp_swp_spi_sf00_L3_VEL_RTN',
 'psp_swp_spi_af00_L3_DENS',
 'psp_swp_spi_af00_L3_VEL_RTN_SUN',
 'psp_swp_spi_af00_L3_VEL_RTN',
 'Tp_spani_b',
 'Tp_spanib_b']

In [6]:
density_col = "plasma_density"
velocity_cols = ["v_R", "v_T", "v_N"]

mag_data = get_data_lf(PSP_MAG_TNAME)
density_data = get_data_lf(PSP_DEN_TNAME).rename({"0": density_col})
velocity_data = get_data_lf(PSP_VEL_TNAME).rename({"0": "v_R", "1": "v_T", "2": "v_N"})
ion_temp_data = get_data_lf(PSP_TEMP_TNAME).rename(
    {"0": "ion_temp_para", "1": "ion_temp_perp"}
)

plasma_data = density_data.join(velocity_data, on="time")

plasma_meta = PlasmaDataset(
    density_col=density_col,
    velocity_cols=velocity_cols,
)

In [8]:
time = mag_data.collect().get_column("time").to_numpy()
ts = np.median(np.diff(time)).item()

In [15]:
def remove(list: list, item):
    temp_list = list.copy()
    temp_list.remove(item)
    return temp_list

In [21]:
from space_analysis.meta import MagDataset

mag_meta = MagDataset(B_cols=remove(mag_data.columns, "time"))

ids_dataset = IDsDataset(
    name=name,
    mag_data=mag_data,
    mag_meta=mag_meta,
    tau=tau,
    ts=ts,
    ion_temp_data=ion_temp_data,
    plasma_data=plasma_data,
    plasma_meta=plasma_meta,
)
ids_dataset.find_events().update_events()

ids_dataset.events = ids_dataset.events.pipe(calc_pressure_anisotropy)

04-Jul-24 06:06:54: DeprecationWarning: invalid escape sequence '\m'

04-Jul-24 06:06:54: DeprecationWarning: invalid escape sequence '\c'

04-Jul-24 06:06:54: DeprecationWarning: invalid escape sequence '\S'

04-Jul-24 06:06:54: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/

04-Jul-24 06:06:55: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.



Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

04-Jul-24 06:06:55: Using sequential splitting in '.from_pandas()' because of some of the conditions are False: enough_elements=False; all_numeric_types=False; async_mode_on=False


Estimated completion of line 15:   0%           Elapsed time: 00:00, estimated remaining time: ?

Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

04-Jul-24 06:06:59: Using sequential splitting in '.from_pandas()' because of some of the conditions are False: enough_elements=False; all_numeric_types=False; async_mode_on=False


Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

04-Jul-24 06:06:59: Using sequential splitting in '.from_pandas()' because of some of the conditions are False: enough_elements=False; all_numeric_types=True; async_mode_on=False


Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

04-Jul-24 06:06:59: Using sequential splitting in '.from_pandas()' because of some of the conditions are False: enough_elements=False; all_numeric_types=False; async_mode_on=False
04-Jul-24 06:07:00: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.

04-Jul-24 06:07:00: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 val

In [ ]:
path = f"../data/enc{enc}/events.PSP.arrow"
ids_dataset.export(path)

ids_dataset